In [88]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import GaussianNB

The dataset contains two columns:
- category: the category of the email
- text: the text of the email

The dataset is available at [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection).

We will read the dataset into a pandas dataframe then specify the category column as the label and the text column as the feature.



In [89]:
df = pd.read_csv('data/spam.csv')[['category', 'text']]
X, y = df['text'].values, df['category']

In [90]:
df.shape

(5572, 2)

We will use the TFIDF algorithm to vectorize the emails. The 3000 best features are selected using the chi-squared test.

In [91]:
K = 3000
tfidf = TfidfVectorizer(analyzer='word')
X = tfidf.fit_transform(X).toarray()
X = SelectKBest(chi2, k=K).fit_transform(X,y)
X = pd.DataFrame(X, columns=tfidf.get_feature_names_out()[0:K])

Now, we have a dataset with 3000 features, and we can use it to train a model.

In [92]:
X.shape

(5572, 3000)

In [93]:
pd.DataFrame(X).describe()

We will use the Naive Bayes classifier to train the model. We will use k-fold cross validation to test the model with k=10.

In [ ]:
model = GaussianNB()
kf = KFold(n_splits=10, random_state=1, shuffle=True)

For each fold of the cross validation, we will train the model on the training set and test the model on the test set. We will calculate the accuracy and the root mean squared of the results of each fold. Finally, we will calculate the average of the root mean squared of the results of each fold.

In [ ]:
scores = {'rmse': [], 'accuracy': []}
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results = [0 if t == p else 1 for t, p in zip(y_test, y_pred)]
    scores['rmse'] += [(np.mean(results)  ** 0.5)]
    scores['accuracy'] += [len([i for i in results if i == 0])/len(results)]
print('RMSE:', '%.3f +/- %.3f' %(np.mean(scores['rmse']), np.std(scores['rmse'])))
print('Accuracy:', '%.3f +/- %.3f' %(np.mean(scores['accuracy']), np.std(scores['accuracy'])))

RMSE: 0.157 +/- 0.031
Accuracy: 0.974 +/- 0.010
